# SOM - CESM Member 001
## V6 - Monthly/Yearly split

#### Template from 'Basic uses of SOMPY library', modified for our use
* http://www.vahidmoosavi.com
* https://github.com/sevamoo/sompy

#### Notebook Features
* Import packages required for SOM module SOMPY, and SOMPY itself (note this must be cloned from repo into local dir
* Load Data
* Clean & organize data into format/interval of interest
* Quick visualizations for context
* 

In [2]:
import matplotlib
#import matplotlib.pyplot as plt
import matplotlib.pylab as plt
%matplotlib inline

import pandas as pd
import numpy as np
from time import time
import sompy

from sklearn.preprocessing import StandardScaler

import os
import xarray as xr

import datetime

import math
import glob

from sklearn.externals import joblib
import random

from sompy.sompy import SOMFactory
from sompy.visualization.plot_tools import plot_hex_map
import logging

import random

backend module://ipykernel.pylab.backend_inline version unknown


In [4]:
os.getcwd()

'/Users/cg/co2/ocean-co2-absorption/notebooks/SOM'

In [5]:
#Windows
#DATA_DIR = 'C:\\Users\\goyetc\\ocean-co2-absorption\\data'

#Mac
DATA_DIR = '/Users/cg/co2/'

#Define key names and import .nc data
dataset_names = {'pCO2': 'pCO2_2D_mon_CESM001_1x1_198201-201701.nc',
                 'XCO2': 'XCO2_1D_mon_CESM001_native_198201-201701.nc',
                 'SST': 'SST_2D_mon_CESM001_1x1_198201-201701.nc',
                 'SSS': 'SSS_2D_mon_CESM001_1x1_198201-201701.nc',
                 'MLD': 'MLD_2D_mon_CESM001_1x1_198201-201701.nc',
                 'Chl': 'Chl_2D_mon_CESM001_1x1_198201-201701.nc'}
ds = {}
for dataset in dataset_names.keys():
    filename = os.path.join(DATA_DIR, dataset_names[dataset])
    ds[dataset] = xr.open_dataset(filename)

In [6]:
#create dataframe for data prep

df = {}
for dataset in ds.keys():
    # e.g. pCO2
    df[dataset] = ds[dataset][dataset].to_dataframe()
    
    #note np.isfinite eliminates infinite and/or NaN records from dataset 
    df[dataset] = df[dataset][np.isfinite(df[dataset][dataset])].reset_index()


In [7]:
SOM_input = pd.concat([df['SSS'],df['MLD']['MLD'],df['SST']['SST'],df['pCO2']['pCO2']], axis=1)

In [8]:
SOM_input.head()

,time,ylat,xlon,SSS,MLD,SST,pCO2
0,1982-01-16 12:00:00,-77.5,179.5,34.000992,16.192860,1.648732,151.525853
1,1982-01-16 12:00:00,-77.5,180.5,33.941429,15.115437,1.528921,150.330599
2,1982-01-16 12:00:00,-77.5,181.5,33.863464,14.071844,1.350243,148.500409
3,1982-01-16 12:00:00,-77.5,182.5,33.775764,13.072312,1.119088,146.198458
4,1982-01-16 12:00:00,-77.5,183.5,33.691376,12.333377,0.865820,143.769723


In [9]:
#Filter out outliers, SSS
SOM_input = SOM_input.loc[SOM_input['SSS'] > 8]

### Clean/label data so it can more easily be split up over different timeframes

In [12]:
#convert time info into date-time format
SOM_input.set_index(pd.DatetimeIndex(SOM_input['time']))

#create month column
SOM_input['month'] = pd.DatetimeIndex(SOM_input['time']).month

#create year column
SOM_input['year'] = pd.DatetimeIndex(SOM_input['time']).year


In [14]:
Year = {}

for j in range(36):
    year = 1982+j
    Year[str(year)] = SOM_input.loc[SOM_input['year'] == year]

In [15]:
for j in range(36):
    year = 1982+j
    
    Year[str(year)] = Year[str(year)].groupby(['ylat','xlon'], as_index=False).mean().drop(columns='month')

In [38]:
#I'm sure there's a better way to do this, but I felt it was easier than creating an empty dataframe with the right dimensions.. 
Years = pd.DataFrame.from_dict(Year['1982'])
for j in range(35):
    year = 1983+j
    Years = Years.append(Year[str(year)])

In [41]:
Years.head()

,ylat,xlon,SSS,MLD,SST,pCO2,year
0,-77.5,179.5,34.334385,246.338547,-1.317129,259.701614,1982.0
1,-77.5,180.5,34.316010,246.692596,-1.337575,259.877004,1982.0
2,-77.5,181.5,34.291920,246.478256,-1.364652,259.905286,1982.0
3,-77.5,182.5,34.264275,242.469589,-1.397035,259.668704,1982.0
4,-77.5,183.5,34.236290,232.115128,-1.431846,259.176946,1982.0


In [42]:
Years.sample(10)

,ylat,xlon,SSS,MLD,SST,pCO2,year
39853,86.5,258.5,32.998074,58.607651,-1.779255,292.046609,1990.0
40481,88.5,166.5,32.105354,57.401066,-1.768753,343.812266,2015.0
24668,9.5,69.5,34.761921,63.620792,28.167164,383.205725,2017.0
27631,20.5,288.5,36.226368,43.372349,26.947407,367.061445,2002.0
13235,-32.5,36.5,34.351223,45.609955,20.888300,306.781005,1982.0
28696,25.5,241.5,33.872002,52.886093,19.831690,372.234325,1994.0
27398,19.5,283.5,36.103989,53.153694,28.345917,409.101873,2016.0
591,-71.5,271.5,33.198223,47.985428,-1.778633,256.437940,2014.0
5082,-56.5,89.5,33.664635,78.591698,1.636385,339.010766,1993.0
18050,-15.5,237.5,35.720245,54.829971,26.113150,373.659978,1994.0


In [58]:
Years.shape

(1477260, 7)

In [16]:
Year['1982'].head()

,ylat,xlon,SSS,MLD,SST,pCO2,year
0,-77.5,179.5,34.334385,246.338547,-1.317129,259.701614,1982.0
1,-77.5,180.5,34.316010,246.692596,-1.337575,259.877004,1982.0
2,-77.5,181.5,34.291920,246.478256,-1.364652,259.905286,1982.0
3,-77.5,182.5,34.264275,242.469589,-1.397035,259.668704,1982.0
4,-77.5,183.5,34.236290,232.115128,-1.431846,259.176946,1982.0


In [26]:
# confirm expected number of samples per year -- matches spatial-only value from monthly-averaged clusters
# confirm # of years matches expected range
Year['1982'].shape, len(SOM_input.year.unique())

(41035, 7)

# Scale Data

#### choice of normalization or transform? 
* Note.. lessons learned from prior optimization:
* - log transform of MLD improved perf
* - StandardScaler demonstrates same perf as 'var' norm option within SOMPY engine
* - variance-based normalization is best choice based on analysis of other scaling options

In [52]:
ss = StandardScaler().fit_transform(Years[['SSS','SST','MLD','pCO2']])

In [53]:
#tuning parameters
#research suggests M = 5*sqrt(N) is a good choice for number of neurons
M = 5*np.sqrt(ss.shape[0])

#calculate grid dimension (for square grid), as basis for optimization of map size
m=int(np.sqrt(M))

M, m

6077.129256482866

### Notes on SOM tuning
* testing suggests pca is faster than random init but limits # of epochs/trainlen internal to SOM engine
* batch faster than sequential.. only batch implemented currently

In [55]:
map_size = [m,m]
names = ['SSS','SST','MLD','pCO2']
 
sm = sompy.SOMFactory().build(ss, mapsize = map_size, mapshape = 'planar',
                            initialization='pca',
                            normalization = 'var', 
                            component_names=names, lattice='rect') 
sm.train(n_job=1, 
         verbose='info', 
         train_rough_len= 10,
         train_finetune_len= 20,
         #train_rough_radiusin=14,
         #train_rough_radiusfin=3.5,
         #train_finetune_radiusin=3.5,
         #train_finetune_radiusfin=1
        ) 
    
joblib.dump(sm, "Yearly_means_combined.joblib")

 Training...
 pca_linear_initialization took: 1.989000 seconds
 Rough training...
 radius_ini: 10.000000 , radius_final: 2.500000, trainlen: 10

 epoch: 1 ---> elapsed time:  163.546000, quantization error: 0.940985

 epoch: 2 ---> elapsed time:  175.621000, quantization error: 0.711951

 epoch: 3 ---> elapsed time:  193.419000, quantization error: 0.595589

 epoch: 4 ---> elapsed time:  193.727000, quantization error: 0.543979

 epoch: 5 ---> elapsed time:  178.715000, quantization error: 0.501750

 epoch: 6 ---> elapsed time:  195.326000, quantization error: 0.462497

 epoch: 7 ---> elapsed time:  193.252000, quantization error: 0.423668

 epoch: 8 ---> elapsed time:  191.145000, quantization error: 0.384678

 epoch: 9 ---> elapsed time:  152.292000, quantization error: 0.345566

 epoch: 10 ---> elapsed time:  149.878000, quantization error: 0.306658

 Finetune training...
 radius_ini: 2.500000 , radius_final: 1.000000, trainlen: 20

 epoch: 1 ---> elapsed time:  151.708000, quantiza

['monthly_means_with_year_feature.joblib']

### From Original SOMtoolbox documentation
* http://www.cis.hut.fi/somtoolbox/package/docs2/som_quality.html

*     qe : Average distance between each data vector and its BMU.
       Measures map resolution.
*     te : Topographic error, the proportion of all data vectors
       for which first and second BMUs are not adjacent units.
       Measures topology preservation.

In [56]:
# Observe results
# note this works to compare multiple SOM training runs...
# but also can just produce results from single training session. 
# use * wildcard, e.g., './Model_xxx_*', to add multiple models to pool for results eval
models_pool_r3 = glob.glob("./Yearly_means_combined.joblib")
errors=[]
for model_filepath in models_pool_r3:
    sm_r3 = joblib.load(model_filepath)
    topographic_error = sm_r3.calculate_topographic_error()
    quantization_error = sm_r3.calculate_quantization_error()
    errors.append((topographic_error, quantization_error))
e_top_r3, e_q_r3 = zip(*errors)

In [ ]:
plt.scatter(e_top_r3, e_q_r3)
plt.title(" ")
plt.xlabel("Topographic error")
plt.ylabel("Quantization error")
plt.show()

In [57]:
res = []
for i, model_filepath in enumerate(models_pool_r3):
    res.append({'model': model_filepath, 'topographical_error': e_top_r3[i], 'quantization_error': e_q_r3[i]})
    #print(model_filepath)
    #name.append(model_filepath[3:10])
    #err_top.append(e_top[i])
    #err_quant.append(e_q[i])

results_df = pd.DataFrame(res)
results_df.sort_values(['topographical_error'], ascending=True)
#print(model_filepath,e_top[i],e_q[i])

,model,quantization_error,topographical_error
0,./monthly_means_with_year_feature.joblib,0.069619,0.128452


In [59]:
som = joblib.load('./Yearly_means_combined.joblib')
map_labels_k5 = som.cluster(n_clusters=5)
data_labels_k5 = np.array([map_labels_k5[int(k)] for k in som._bmu[0]])
map_labels_k10 = som.cluster(n_clusters=10)
data_labels_k10 = np.array([map_labels_k10[int(k)] for k in som._bmu[0]])
map_labels_k15 = som.cluster(n_clusters=15)
data_labels_k15 = np.array([map_labels_k15[int(k)] for k in som._bmu[0]])

Years['k=5'] = data_labels_k5
Years['k=10'] = data_labels_k10
Years['k=15'] = data_labels_k15

Years.to_csv('./Yearly.csv',
              sep=',',
              na_rep='NaN',
              columns=[u'ylat', u'xlon', u'SSS', u'MLD', u'SST', u'pCO2', u'year', u'k=5', u'k=10', u'k=15'],
              header=True,
              index=True,
              mode='w')

In [60]:
sm_r3.calculate_topographic_error()

0.12845199897106807

In [61]:
Years.sample(10)

,ylat,xlon,SSS,MLD,SST,pCO2,year,k=5,k=10,k=15
31299,38.5,233.5,32.638065,47.570847,14.123396,356.257071,1999.0,2,7,12
39043,84.5,168.5,32.260899,61.237625,-1.777910,307.225312,1995.0,0,1,10
3513,-61.5,320.5,33.929203,67.054085,0.159267,358.388573,2015.0,2,4,0
21382,-3.5,335.5,35.381069,64.442390,26.578503,384.626153,2000.0,1,2,7
30192,33.5,15.5,41.909344,48.425957,20.532146,387.688482,2003.0,1,9,4
16052,-22.5,81.5,34.796764,69.243813,23.015705,323.108072,1985.0,1,0,13
22667,1.5,243.5,34.526443,73.657753,26.815346,434.706510,2002.0,1,2,11
19952,-8.5,190.5,34.314548,80.206612,28.578405,365.503266,2012.0,1,8,7
1284,-67.5,19.5,33.301205,24.069923,-1.641757,337.571563,2017.0,0,1,0
25753,13.5,116.5,32.047298,45.659595,27.443811,333.598754,1994.0,1,8,1
